<a href="https://colab.research.google.com/github/deltorobarba/science/blob/main/alicebob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####################################
# Quantum Computing on Alice & Bob
####################################

!pip install -U qiskit-alice-bob-provider -q
from google.colab import userdata
from qiskit_alice_bob_provider import AliceBobRemoteProvider
from qiskit import QuantumCircuit, transpile

# 1. Retrieve API key
api_token = userdata.get('alicebob')

# 2. Connect to the Alice & Bob
provider = AliceBobRemoteProvider(api_key=api_token)

# 3. List available backends
for backend in provider.backends():
    print(backend.name)

# 4. Select backend (QPU quantum or EMU emulator)
backend = provider.get_backend('EMU:6Q:PHYSICAL_CATS', average_nb_photons=10.0)
print(f"\nSelected backend: {backend.name}")

# 5. Define a simple quantum circuit
qc = QuantumCircuit(1, 1)
qc.initialize('1', 0)
qc.delay(duration=1, qarg=0, unit='ms')
qc.measure(0, 0) # Measures bit-flip probability of single qubit after a 1ms delay
print(qc.draw())

# 6. Transpile circuit to specific backend to ensure correct error-correction properties
transpiled = transpile(qc, backend)

# 7. Run circuit on Felis Cloud
job = backend.run(transpiled, shots=1000)
res = job.result()
print(f'Total counts: {res.get_counts()}')

# Console: https://api-gcp.alice-bob.com/console
# Backend: https://felis.alice-bob.com/docs/backends/about_backends/
# Status:  https://felis.alice-bob.com/docs/felis_cloud/hardware_availability_schedule/
# Samples: https://github.com/Alice-Bob-SW/felis/tree/main/samples

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 9.9 MB/s eta 0:00:00
EMU:1Q:LESCANNE_2020
QPU:1Q:BOSON_4A
QPU:1Q:BOSON_4B
QPU:1Q:BOSON_4C
EMU:6Q:PHYSICAL_CATS
EMU:40Q:PHYSICAL_CATS
EMU:40Q:LOGICAL_TARGET
EMU:40Q:LOGICAL_NOISELESS
EMU:15Q:LOGICAL_EARLY

Selected backend: EMU:6Q:PHYSICAL_CATS
     ┌───────────────┐┌──────────────┐┌─┐
  q: ┤ Initialize(1) ├┤ Delay(1[ms]) ├┤M├
     └───────────────┘└──────────────┘└╥┘
c: 1/═══════════════════════

Label(value='')

Total counts: {'1': 1000}
